# **Import libraries** 

In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Initialize variables

In [ ]:
img_size = 128
epochs = 5
batch_size = 32
train_path = '/content/drive/MyDrive/archive'
folders = os.listdir(train_path)
train_df = pd.DataFrame(columns=['filename', 'class'])

# Load the image filenames and labels into a dataframe

In [ ]:
for folder in folders:
    path = os.path.join(train_path, folder)
    filenames = os.listdir(path)
    for filename in filenames:
        img_path = os.path.join(path, filename)
        train_df = train_df.append({'filename': img_path, 'class': folder}, ignore_index=True)

<ipython-input-4-4603eb5f6410>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'filename': img_path, 'class': folder}, ignore_index=True)
<ipython-input-4-4603eb5f6410>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'filename': img_path, 'class': folder}, ignore_index=True)
<ipython-input-4-4603eb5f6410>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'filename': img_path, 'class': folder}, ignore_index=True)
<ipython-input-4-4603eb5f6410>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'filename': img_path, 'class': fold

# Split the data into training and validation sets

In [ ]:
train_df = train_df.sample(frac=1, random_state=42)
train_len = int(len(train_df) * 0.8)
train_data = train_df[:train_len]
valid_data = train_df[train_len:]

# Create generators for training and validation data 

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='filename',
    y_col='class',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_data,
    x_col='filename',
    y_col='class',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

Found 544 validated image filenames belonging to 2 classes.
Found 137 validated image filenames belonging to 2 classes.


#  Load the VGG19 model

In [ ]:
vgg = VGG19(input_shape=(128, 128, 3), weights='imagenet', include_top=False)

# **Freeze the layers in VGG19**

In [ ]:
for layer in vgg.layers:
  layer.trainable = False

# Build the Model

In [ ]:
x = Flatten()(vgg.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

# Compile the model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

#  Train the model

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    steps_per_epoch=len(train_data)//batch_size,
    validation_steps=len(valid_data)//batch_size)

Epoch 1/5
17/17 [==============================] - 243s 14s/step - loss: 9.9090 - accuracy: 0.7206 - val_loss: 0.6641 - val_accuracy: 0.9219
Epoch 2/5
17/17 [==============================] - 206s 12s/step - loss: 0.3815 - accuracy: 0.9412 - val_loss: 0.1844 - val_accuracy: 0.9531
Epoch 3/5
17/17 [==============================] - 198s 12s/step - loss: 0.2433 - accuracy: 0.9449 - val_loss: 0.3032 - val_accuracy: 0.9062
Epoch 4/5
17/17 [==============================] - 206s 12s/step - loss: 0.2369 - accuracy: 0.9485 - val_loss: 0.3954 - val_accuracy: 0.8828
Epoch 5/5
17/17 [==============================] - 207s 12s/step - loss: 0.1554 - accuracy: 0.9559 - val_loss: 0.2343 - val_accuracy: 0.9141


## Evaluate the model

In [ ]:
test_generator = valid_datagen.flow_from_dataframe(
    valid_data,
    x_col='filename',
    y_col='class',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 137 validated image filenames belonging to 2 classes.


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


5/5 [==============================] - 40s 7s/step - loss: 0.2612 - accuracy: 0.9051
Test accuracy: 0.9051094651222229


## Save the model

In [ ]:
model.save('VGG19.h5')

# Predictions

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the pre-trained VGG19 model
model = tf.keras.applications.VGG19(weights='imagenet')

# Load and preprocess the image
img_path = 'path/to/image.jpg'
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make predictions
preds = model.predict(x)
top_preds = decode_predictions(preds, top=5)[0]

# Print the top predictions
for pred in top_preds:
    print(f"{pred[1]}: {pred[2]*100:.2f}%")
